In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from src.models import ResNet
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
num_epochs = 80
batch_size = 100
learning_rate = 0.001
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)
#stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))


# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# Normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(*stats)
])

# CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform_train, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform_test)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified


# Visualize some of the training data

In [40]:
def denormalize(images, means, stds):
    means = torch.tensor(means).reshape(1, 3, 1, 1)
    stds = torch.tensor(stds).reshape(1, 3, 1, 1)
    return images * stds + means


figure = plt.figure()
num_of_images = 20
for index in range(1, num_of_images + 1):
        image, labels = train_dataset[index]
        plt.subplot(60, 20, index)
        plt.axis('off')
        #fig, ax = plt.subplots(figsize=(12, 12))
        #ax.set_xticks([]); ax.set_yticks([])
        denorm_image = denormalize(image, *stats)
        #ax.imshow(make_grid(denorm_image, nrow=8).permute(1, 2, 0).clamp(0,1))
        plt.imshow(denorm_image.permute(1, 2, 0,3).numpy().squeeze())



TypeError: Invalid shape (3, 32, 32) for image data

In [4]:
resnet = ResNet(in_channels=16, num_classes=10)
resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bia

In [5]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)


In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Save the model checkpoint
torch.save(resnet.state_dict(), 'resnet.ckpt')


Epoch [1/80], Step [100/500], Loss: 1.6597
Epoch [1/80], Step [200/500], Loss: 1.5143
Epoch [1/80], Step [300/500], Loss: 1.3115
Epoch [1/80], Step [400/500], Loss: 1.1933
Epoch [1/80], Step [500/500], Loss: 1.2585
Epoch [2/80], Step [100/500], Loss: 1.0379
Epoch [2/80], Step [200/500], Loss: 1.2477
Epoch [2/80], Step [300/500], Loss: 0.9151
Epoch [2/80], Step [400/500], Loss: 1.1285
Epoch [2/80], Step [500/500], Loss: 0.9303
Epoch [3/80], Step [100/500], Loss: 1.0973
Epoch [3/80], Step [200/500], Loss: 0.7247
Epoch [3/80], Step [300/500], Loss: 0.9803
Epoch [3/80], Step [400/500], Loss: 0.8689
Epoch [3/80], Step [500/500], Loss: 0.6966
Epoch [4/80], Step [100/500], Loss: 0.6708
Epoch [4/80], Step [200/500], Loss: 0.7304
Epoch [4/80], Step [300/500], Loss: 0.5850
Epoch [4/80], Step [400/500], Loss: 0.7302
Epoch [4/80], Step [500/500], Loss: 0.7242
Epoch [5/80], Step [100/500], Loss: 0.6566
Epoch [5/80], Step [200/500], Loss: 0.6212
Epoch [5/80], Step [300/500], Loss: 0.7382
Epoch [5/80

In [9]:
# Test the model
model_dict = torch.load('resnet.ckpt')
model = ResNet(in_channels=16, num_classes=10)
model.load_state_dict(model_dict)
model.to(device)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 88.26 %
